# How to Implement Logistic Regression with PyTorch
![alt text](imgs/lr.png)

In [1]:
import torch

To understand better what we’re going to do next, you can read [this article about logistic regression](https://towardsdatascience.com/understanding-logistic-regression-81779525d5c6).  
So, what's our plan for implementing Logistic Regression with PyTorch?  
Let's first think of the underlying math that we want to use.  
There are many ways to define a loss function and then find the optimal parameters for it, among them, here we will implement in our `LogisticRegression` class the following 3 ways for learning the parameters:
- We will rewrite the logistic regression equation so that we turn it into a least-squares linear regression problem with different labels and then, we use the closed-form formula to find the weights: ![at text](imgs/eq1.png)  
- Like above, we turn logistic into least-squares linear regression, but instead of the closed-form formula, we use stochastic gradient descent (SGD) to minimize the following loss function: ![alt text](imgs/eq4.png) which was obtained by substituting the y in the sum of squared errors loss ![alt text](imgs/eq3.png) with the right-hand side of ![alt text](imgs/eq2.png)  
- We use the maximum likelihood estimation (MLE) method, write the likelihood function, play around with it, restate it as a minimization problem, and apply SGD with the following loss function: ![alt text](imgs/eq5.png)

In the above equations, X is the input matrix that contains observations on the row axis and features on the column axis; y is a column vector that contains the classification labels (0 or 1); f is the sum of squared errors loss function; h is the loss function for the MLE method.   
If you want to find out more about how we obtained the above equations, please check out the above-linked article.   
So now, this is our goal: translate the above equations into code. And we’ll use PyTorch for that.   
We plan to use an object-oriented approach for implementation. We'll create a `LogisticRegression` class with 3 public methods: `fit()`, `predict()`, and `accuracy()`.   
Among fit's parameters, one will determine how our model learns. This parameter is named method (not to be confused with a method as a function of a class) and it can take the following strings as values: 'ols_solve' (OLS stands for Ordinary Least Squares), 'ols_sgd’, and 'mle_sgd’.   
To not make the `fit()` method too long, we would like to split the code into 3 different private methods, each one responsible for one way of finding the parameters.   
We will have the `__ols_solve()` private method for applying the closed-form formula.  
In this method and in the other methods that use the OLS approach, we will use the constant EPS to make sure the labels are not exactly 0 or 1, but something in between. That’s to avoid getting plus or minus infinity for the logarithm in the equations above.   
In `__ols_solve()` we first check if X has full column rank so that we can apply this method (you can read more about this techinque and what happens if X doesn't have full column rank in [this article](https://towardsdatascience.com/understanding-linear-regression-eaaaed2d983e)). Then we force y to be between EPS and 1-EPS. The `ols_y` variable holds the labels of the ordinary least-squares linear regression problem that’s equivalent to our logistic regression problem. Basically, we transform the labels that we have for logistic regression so that they are compliant with the linear regression equations. After that, we apply the closed-form formula using PyTorch functions. 

```python
EPS = 1e-5
def __ols_solve(self, x, y):
    rows, cols = x.shape
    if rows >= cols == torch.matrix_rank(x):
        eps = self.EPS * torch.ones_like(y)
        y = torch.max(eps, torch.min(y.float(), 1-eps))
        ols_y = -torch.log(torch.div(1.0, y) - 1)

        xt = torch.transpose(x, 0, 1)
        self.weights = torch.matmul(
            torch.matmul(
                torch.inverse(
                    torch.matmul(xt, x)
                ),
                xt),
            ols_y)
    else:
        print('Error! X has not full column rank.')
```

For the 2 SGD-based algorithms, it would be redundant to have them as 2 separate methods since they will have almost all the code the same except for the part where we compute the loss value, as we have 2 different loss functions for them.   
What we’ll do is to create a generic `__sgd()` method that does not rely on a particular loss function. Instead, it will expect as a parameter a function responsible for computing the loss value which the `__sgd()` method will use.   
In this method, we first initialize the weights to a random column vector with values drawn from a normal distribution with mean 0 and a standard deviation of 1/(# of features). The intuition for this std dev is that if we have more features, then we need smaller weights to be able to converge (and not blow up our gradients).   
We will create a `DataLoader` object with `shuffle=True` so that it will take care of shuffling and simply return us a batch of data `(xb, yb)` when we iterate over it with `for step, (xb, yb) in enumerate(loader):`.   
Then we go through all the dataset for `iterations` times and for each batch of data, we compute the loss value using the `loss_fn` function taken as a parameter, then we call the `backward()` method on the loss value.   
By calling `backward()`, the gradient is computed and stored in the `grad` attribute of `self.weights`. So next we can use `self.weights -= learning_rate * self.weights.grad` to update the weights, and we do so inside the `with torch.no_grad():` block because we don't want this operation of updating the weights to be considered the next time we call `backward()`.   
By default, PyTorch keeps track of every operation that involves a tensor which has `requires_grad == True` (in our case, `self.weights`), and when `backward()` is called, the gradient of the whole function composition chain is computed. The operations that we don't want in the gradient computation we can put them inside a `with torch.no_grad():` block.   
An important thing to note about automatic differentiation in PyTorch, is that when `backward()` is called, **the gradient is NOT set anew**, but instead **the new gradient is added to the existing value in the `grad` attribute**.   
That's why we have to set `self.weights.grad` to zero after each update, and we do so by using `self.weights.grad.zero_()`.    
Then, after the training is done, we use `self.weights = self.weights.detach()` to detach `self.weights` from the computational graph, which means that it will not require grad anymore.

```python
def __sgd(self, x, y, loss_fn, learning_rate, iterations, batch_size):
    rows, cols = x.shape
    self.weights = torch.normal(mean=0, std=1/cols, size=(cols, 1), requires_grad=True)
    dataset = torch.utils.data.TensorDataset(x, y)
    loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

    for i in range(iterations):
        for step, (xb, yb) in enumerate(loader):
            loss = loss_fn(xb, yb)
            loss.backward()

            with torch.no_grad():
                self.weights -= learning_rate * self.weights.grad
                self.weights.grad.zero_()

    self.weights = self.weights.detach()
```

For ‘ols_sgd’ and ‘mle_sgd’ we’ll create 2 private methods: `__sse_loss()` and `__mle_loss()` that compute and return the loss value for these 2 different techniques.   
For these 2 methods, we simply apply the formulas for **f** and **h** using PyTorch’s math functions.   
```python
def __sse_loss(self, xb, yb):
    eps = self.EPS * torch.ones_like(yb)
    yb = torch.max(eps, torch.min(yb, 1-eps))
    ols_yb = -torch.log(torch.div(1.0, yb) - 1)

    diff = torch.matmul(xb, self.weights) - ols_yb
    loss = torch.matmul(torch.transpose(diff, 0, 1), diff)

    return loss

def __mle_loss(self, xb, yb):
    xw = torch.matmul(xb, self.weights)
    term1 = torch.matmul(torch.transpose(1-yb, 0, 1), xw)
    term2 = torch.matmul(
        torch.transpose(torch.ones_like(yb, dtype=torch.float32), 0, 1),
        torch.log(1+torch.exp(-xw)))
    return term1+term2
```

So, when `fit()` is called with `method=‘ols_solve’` we call `__ols_solve()`, when `method=‘ols_sgd’` we call `__sgd()` with `loss_fn=self.__sse_loss`, and when `method=’mle_sgd’` we call `__sgd()` with `loss_fn=self.__mle_loss`.   
```python
def fit(self, x, y, method, learning_rate=0.001, iterations=500, batch_size=32):
    x, y = torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)
    x = torch.cat([x, torch.ones_like(y, dtype=torch.float32)], dim=1)
    if method == "ols_solve":
        self.__ols_solve(x, y)
    elif method == "ols_sgd":
        self.__sgd(x, y, self.__sse_loss, learning_rate, iterations, batch_size)
    elif method == "mle_sgd":
        self.__sgd(x, y, self.__mle_loss, learning_rate, iterations, batch_size)
    else:
        print(f'Unknown method: \'{method}\'')

    return self
```

In `predict()` we first check if `fit()` was called previously by looking for the `weights` attribute (the fit method is the only method that creates it). Then we check if the shapes of the input matrix x and weights vector allow multiplication. Otherwise, return error messages. If everything is OK, we do the multiplication and pass the result through the logistic function.   
```python
def predict(self, x):
    if not hasattr(self, 'weights'):
        print('Cannot predict. You should call the .fit() method first.')
        return

    x = torch.tensor(x, dtype=torch.float32)
    x = torch.cat([x, torch.ones(size=(x.shape[0], 1), dtype=torch.float32)], dim=1)

    if x.shape[1] != self.weights.shape[0]:
        print(f'Shapes do not match. {x.shape[1]} != {self.weights.shape[0]}')
        return

    xw = torch.matmul(x, self.weights)
    return torch.div(1, 1+torch.exp(-xw))
```

In `accuracy()` we make predictions using the above method. Then check if the shape of the predictions matches that of the true labels, otherwise, we show an error message. After that we make sure that both predictions and the true labels have values of either 0 or 1 by a simple rule: if the value is >= 0.5 consider it a 1, otherwise a 0. We apply this rule by using the `torch.where()` function.    
To compute the accuracy, we check for equality between y and y_hat. This will return a vector of Boolean values. Then cast these Booleans to float (False becomes 0.0, and True becomes 1.0). Then, the accuracy is simply the mean of these values.   
```python
def accuracy(self, x, y):
    y_hat = self.predict(x)

    if y.shape != y_hat.shape:
        print('Error! Predictions don\'t have the same shape as given y')
        return

    x, y = torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)
    zeros, ones = torch.zeros_like(y), torch.ones_like(y)
    y = torch.where(y >= 0.5, ones, zeros)
    y_hat = torch.where(y_hat >= 0.5, ones, zeros)

    return torch.mean((y == y_hat).float())
```

Here is the full code of the `LogisticRegression` class:

In [2]:
class LogisticRegression:
    EPS = 1e-5
    def __ols_solve(self, x, y):
        rows, cols = x.shape
        if rows >= cols == torch.matrix_rank(x):
            eps = self.EPS * torch.ones_like(y)
            y = torch.max(eps, torch.min(y.float(), 1-eps))
            ols_y = -torch.log(torch.div(1.0, y) - 1)
            
            xt = torch.transpose(x, 0, 1)
            self.weights = torch.matmul(
                torch.matmul(
                    torch.inverse(
                        torch.matmul(xt, x)
                    ),
                    xt),
                ols_y)
        else:
            print('Error! X has not full column rank.')
    
    def __sgd(self, x, y, loss_fn, learning_rate, iterations, batch_size):
        rows, cols = x.shape
        self.weights = torch.normal(mean=0, std=1/cols, size=(cols, 1), requires_grad=True)
        dataset = torch.utils.data.TensorDataset(x, y)
        loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
        
        for i in range(iterations):
            for step, (xb, yb) in enumerate(loader):
                loss = loss_fn(xb, yb)
                loss.backward()
                
                with torch.no_grad():
                    self.weights -= learning_rate * self.weights.grad
                    self.weights.grad.zero_()
        
        self.weights = self.weights.detach()
    
    def __sse_loss(self, xb, yb):
        eps = self.EPS * torch.ones_like(yb)
        yb = torch.max(eps, torch.min(yb, 1-eps))
        ols_yb = -torch.log(torch.div(1.0, yb) - 1)
        
        diff = torch.matmul(xb, self.weights) - ols_yb
        loss = torch.matmul(torch.transpose(diff, 0, 1), diff)
        
        return loss
    
    def __mle_loss(self, xb, yb):
        xw = torch.matmul(xb, self.weights)
        term1 = torch.matmul(torch.transpose(1-yb, 0, 1), xw)
        term2 = torch.matmul(
            torch.transpose(torch.ones_like(yb, dtype=torch.float32), 0, 1),
            torch.log(1+torch.exp(-xw)))
        return term1+term2
    
    def fit(self, x, y, method, learning_rate=0.001, iterations=500, batch_size=32):
        x, y = torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)
        x = torch.cat([x, torch.ones_like(y, dtype=torch.float32)], dim=1)
        if method == "ols_solve":
            self.__ols_solve(x, y)
        elif method == "ols_sgd":
            self.__sgd(x, y, self.__sse_loss, learning_rate, iterations, batch_size)
        elif method == "mle_sgd":
            self.__sgd(x, y, self.__mle_loss, learning_rate, iterations, batch_size)
        else:
            print(f'Unknown method: \'{method}\'')
        
        return self
    
    def predict(self, x):
        if not hasattr(self, 'weights'):
            print('Cannot predict. You should call the .fit() method first.')
            return
        
        x = torch.tensor(x, dtype=torch.float32)
        x = torch.cat([x, torch.ones(size=(x.shape[0], 1), dtype=torch.float32)], dim=1)
        
        if x.shape[1] != self.weights.shape[0]:
            print(f'Shapes do not match. {x.shape[1]} != {self.weights.shape[0]}')
            return
        
        xw = torch.matmul(x, self.weights)
        return torch.div(1, 1+torch.exp(-xw))
    
    def accuracy(self, x, y):
        y_hat = self.predict(x)
        
        if y.shape != y_hat.shape:
            print('Error! Predictions don\'t have the same shape as given y')
            return
        
        x, y = torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)
        zeros, ones = torch.zeros_like(y), torch.ones_like(y)
        y = torch.where(y >= 0.5, ones, zeros)
        y_hat = torch.where(y_hat >= 0.5, ones, zeros)
        
        return torch.mean((y == y_hat).float())

Now we want to see how our `LogisticRegression` class performs on this heart disease dataset.

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [4]:
df = pd.read_csv('heart.csv')
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


In [5]:
x, y = df.iloc[:, 0:-1].values, df.iloc[:, -1].values.reshape((-1, 1))
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8)

In [6]:
scaler = MinMaxScaler().fit(x_train)
x_train, x_test = scaler.transform(x_train), scaler.transform(x_test)

In [7]:
def print_acc(model):
    print(f'Train accuracy = {model.accuracy(x_train, y_train)} ; '+
          f'Test accuracy = {model.accuracy(x_test, y_test)}')

### Using 'ols_solve' method

In [8]:
lr_ols_solve = LogisticRegression().fit(x_train, y_train, 'ols_solve')
print_acc(lr_ols_solve)

Train accuracy = 0.8429751992225647 ; Test accuracy = 0.8360655903816223


### Using 'ols_sgd' method

In [9]:
lr_ols_sgd = LogisticRegression().fit(x_train, y_train, 'ols_sgd')
print_acc(lr_ols_sgd)

Train accuracy = 0.8429751992225647 ; Test accuracy = 0.8360655903816223


### Using 'mle_sgd' method

In [10]:
lr_mle_sgd = LogisticRegression().fit(x_train, y_train, 'mle_sgd')
print_acc(lr_mle_sgd)

Train accuracy = 0.8512396812438965 ; Test accuracy = 0.8032786846160889
